In [1]:
import pdfplumber
import pandas as pd

import re

from csv import writer

# Convert PDF to CSV

In [2]:
# Open PDF file
pdf = pdfplumber.open("data/sao-ke-092024.pdf")

In [3]:
# Check number of pages
len(pdf.pages)

12028

In [ ]:
page_0 = pdf.pages[1]

sample = page_0.extract_text_simple()

sample

In [ ]:
VALID_START = "Doc No"

VALID_END = "\nPostal address"

valid_sample = sample.split(VALID_START)[1].split(VALID_END)[0]

valid_sample

In [6]:
SPLIT_PATTERN = r'\n\d{2}/\d{2}/\d{4}'

In [ ]:
parts = re.split(f'({SPLIT_PATTERN})', valid_sample)

parts

In [ ]:
len(parts)

In [ ]:
substrings = [(parts[i] + parts[i+1]).strip() for i in range(1, len(parts)-1, 2) if parts[i].strip()]

substrings

In [ ]:
# Get Date
date_pattern = r'\b\d{2}/\d{2}/\d{4}\b'
re.search(date_pattern, substrings[0]).group(0)

In [ ]:
# Get Amount
substrings[0].split("\n", 1)[1].split()[0]

In [ ]:
# Get Message
substrings[0].split("\n", 1)[1].strip().replace("  ", " ").split(" ", 1)[1]

In [ ]:
timestamp_pattern = r'\n\d{4}\.\d+'
re.sub(timestamp_pattern, '', substrings[14].split("\n", 1)[1].strip().replace("  ", " ").split(" ", 1)[1])

In [4]:
def get_clean_data_row(text):
    # Get Date
    date_pattern = r'\b\d{2}/\d{2}/\d{4}\b'
    date = re.search(date_pattern, text).group(0)

    # Get Amount
    amount = text.split("\n", 1)[1].split()[0]

    # Get Message
    message = text.split("\n", 1)[1].strip().replace("  ", " ").split(" ", 1)[1]
    timestamp_pattern = r'\n\d{4}\.\d+'
    message = re.sub(timestamp_pattern, '', message)

    return [date, amount, message]


def convert_data_to_list(text) -> list:
    SPLIT_PATTERN = r'\n\d{2}/\d{2}/\d{4}'
    parts = re.split(f'({SPLIT_PATTERN})', text)
    sub_samples = [(parts[i] + parts[i+1]).strip() for i in range(1, len(parts)-1, 2) if parts[i].strip()]
    return sub_samples

    
def get_valid_data(text) -> str:
    VALID_START = "Doc No"
    VALID_END = "\nPostal address"

    return text.split(VALID_START)[1].split(VALID_END)[0]

In [ ]:
# Create DataFrame
df = pd.DataFrame(columns=["Date", "Credit", "Message"])

df

In [11]:
df.to_csv("data/sao-ke-092024.csv", encoding="utf-8", index=False)

In [ ]:
for i in range(8791, len(pdf.pages)):
    print(f"===============Processing Page {i+1}===================")

    page = pdf.pages[i]
    text = page.extract_text_simple()

    valid_text = get_valid_data(text)
    rows = convert_data_to_list(valid_text)
    
    for row in rows:
        clean_data = get_clean_data_row(row)

        with open("data/sao-ke-092024.csv", encoding="utf-8", mode="a") as f:
            csv_writer = writer(f)
            csv_writer.writerow(clean_data)
            f.close()

In [2]:
df = pd.read_csv("data/sao-ke-092024.csv", encoding="utf-8")

In [3]:
df.head()

,Date,Credit,Message
0,01/09/2024,50.000,292976.010924.013647.xin cam on
1,01/09/2024,20.000,VCB.CTDK.31/03/2024.ADIDA PHAT. CT tu\r\n04810...
2,01/09/2024,29.000,MBVCB.6916176124.CAO VIET TUAN\r\nchuyen tien....
3,01/09/2024,3.000,272986.010924.101858.DO DUC LOI chuyen\r\ntien
4,01/09/2024,3.000,020097040509011046122024JDC5013867.96713\r\n.1...


In [4]:
df.tail()

,Date,Credit,Message
200360,10/09/2024,500.000,MBVCB.6998628625.Cuu tro thiet hai bao\r\nYagi...
200361,10/09/2024,300.000,884114.100924.223854.DANG DAO THANH\r\nNHAN ch...
200362,10/09/2024,100.000,243304.100924.223855.Ung ho dong bao bi\r\nbao lu
200363,10/09/2024,200.000,884140.100924.223855.Nga Pham chuyen tien\r\nF...
200364,10/09/2024,46.667,807439.100924.223851.cua it long nhieu...


In [20]:
len(df)

200365

In [7]:
df["Credit"].value_counts()

Credit
100.000      43602
200.000      43021
500.000      32081
1.000.000    19626
50.000       14857
             ...  
110.521          1
12.504           1
7.113            1
33.355           1
28.887           1
Name: count, Length: 3629, dtype: int64